In [49]:
#building an ai chatbot using langchain
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [50]:
#calling llm model
from langchain_community.llms import Ollama
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025A93D36010>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025A93D097D0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [51]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, how are you?")])


AIMessage(content="As an AI, I don't have feelings, but I'm here and ready to help! How can I assist you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 15, 'total_tokens': 47, 'completion_time': 0.058181818, 'prompt_time': 8.014e-05, 'queue_time': 0.023186596, 'total_time': 0.058261958}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3369e1a7-9bb8-4ef5-8fcf-020c4e966d2f-0', usage_metadata={'input_tokens': 15, 'output_tokens': 32, 'total_tokens': 47})

In [52]:
from langchain_core.messages import AIMessage
model.invoke([HumanMessage(content="Hello, how are you?"),
         AIMessage(content="As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today?"),
         HumanMessage(content="Hey,What do I do?")
         ])


AIMessage(content="That's a big question!  To help me figure out what you should do, tell me more about what's going on. For example:\n\n* **Are you feeling stuck or bored?** Maybe I can suggest a fun activity or creative project.\n* **Do you have a problem you need help solving?** I can try to brainstorm solutions with you.\n* **Are you just curious about something?** I'm happy to answer your questions to the best of my ability.\n\nThe more information you give me, the better I can understand and help! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 61, 'total_tokens': 185, 'completion_time': 0.225454545, 'prompt_time': 0.004305848, 'queue_time': 0.022802797, 'total_time': 0.229760393}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-600fb5c9-f2bc-4f38-906e-d99ccddcb4ee-0', usage_metadata={'input_tokens': 61, 'output_tokens': 124, 'total_tokens': 185})

In [53]:
##Message History
#we can use message histpry class to wrap our model and make it stateful.This will keep track of inputs and outpts of models and store them in some  datastore
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# it create a session id in form of string and there is a chat history which will distingush between different sessions by having unique session id
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [54]:
config={"configurable":{"session_id":"chat1"}}


In [55]:
response=with_message_history.invoke([HumanMessage(content="Hello, how are you?")],config=config)

In [56]:
response.content

"As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today?\n"

In [57]:
#changing the session id to chat2 if we change the ssion id then it will use different session and use the different information stored in this session_id
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke([HumanMessage(content="Hello, how are you?")],config=config)

In [58]:
response.content

"As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today?\n"

In [59]:
#now we use propmt templates Helps to turn a raw information into a format that LLM can work with in this case the raw user input is just a message which we are passing to the lms that's now with that a bit more complicated first let's add in a system message with some custom instructions


In [60]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a Helpful AI assistant, Answer all the questions to the best of your ability in your ."),
        MessagesPlaceholder(variable_name="messages")


    ]
)

chain=prompt|model

In [61]:
chain.invoke({"messages":[HumanMessage(content="Hello, how are you?")]})

AIMessage(content="Hello! I am doing well, thank you for asking. As an AI, I don't have feelings, but I'm ready to assist you with any questions or tasks you may have.  \n\nHow can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 35, 'total_tokens': 90, 'completion_time': 0.1, 'prompt_time': 0.000319599, 'queue_time': 0.022159678, 'total_time': 0.100319599}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-99a03600-ef28-4d06-afb0-9f184047aff3-0', usage_metadata={'input_tokens': 35, 'output_tokens': 55, 'total_tokens': 90})

In [62]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [63]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hello, how are you?")],  # Wrap in a dictionary
    config=config
)

In [64]:
response.content

"Hello! I'm doing well, thank you for asking.  \n\nHow can I help you today? 😊  \n"

In [65]:
#add more complexity
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a Helpful AI assistant, Answer all the questions to the best of your ability in your {language} ."),
        MessagesPlaceholder(variable_name="messages")


    ]
)

chain=prompt|model

In [66]:
response=chain.invoke({"messages":[HumanMessage(content="Hello, how are you?")],"language":"Hindi"})#here we re passing the language as key
response.content

'नमस्ते! मैं अच्छा हूँ, आप कैसे हैं? \n'

In [67]:
#lets now wrap this more complictaed chain in a message history class as there are multiple keys in the input we need to specify the correct key
with_message_history=RunnableWithMessageHistory(chain, get_session_history,input_messages_key="messages")


In [68]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hello, how are you?")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते! मैं अच्छा हूँ, आप कैसे हैं? 😊 \n'

In [69]:
#managing the coversational histpry 
#we will add the step that limit the size of messages we re passing in 'trim_messages' method used to reduce how many messages we re sending to the model 

In [71]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"

)
messages=[
    SystemMessage(content="you re a good assistant"),
    HumanMessage(content="Hello, how are you?"),
    AIMessage(content="I'm here to help you!"),
    HumanMessage(content="I like chocolate"),
    AIMessage(content="I like chocolate too!"),
    HumanMessage(content="What's 2+2?"),
    AIMessage(content="4"),


]
trimmer.invoke(messages)


c:\Users\aakash\Desktop\LCEL-LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\aakash\Desktop\LCEL-LLM\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aakash\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limit

[SystemMessage(content='you re a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm here to help you!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like chocolate', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I like chocolate too!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's 2+2?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={})]

In [75]:
#in a chain how we pass this particular trimmed 
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model

)
response=chain.invoke({
               "messages": messages +[HumanMessage(content="What's 2+2?")],
                       "language":"English"
                     } )
response.content


'2 + 2 = 4 😊 \n\nIs there anything else I can help you with?\n'